In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install marker-pdf pymupdf torch torchvision --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.9/188.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.2/223.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 113.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 127.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.4/226.4 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 117.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79

In [6]:
!pip install pdfplumber pymupdf --quiet

In [9]:
import os
import json
import pdfplumber
import fitz  # PyMuPDF
from collections import Counter
#Configuration
BASE_PATH = "/content/drive/MyDrive/NCERT_Assignment"
SUBJECTS = {
    "Mathematics": os.path.join(BASE_PATH, "Input_pdf/Math"),
    "Science": os.path.join(BASE_PATH, "Input_pdf/Science")
}
OUTPUT_BASE_DIR = os.path.join(BASE_PATH, "Output_Data")
os.makedirs(OUTPUT_BASE_DIR, exist_ok=True)

#  R.Image Extractor (CMYK FIXED)
def extract_images_robust(pdf_path, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    doc = fitz.open(pdf_path)
    count = 0

    for page_idx in range(len(doc)):
        for img_idx, img in enumerate(doc.get_page_images(page_idx)):
            xref = img[0]
            try:
                pix = fitz.Pixmap(doc, xref)

                # Convert CMYK (4+ channels) to RGB (3 channels)
                if pix.n - pix.alpha >= 4:
                    pix = fitz.Pixmap(fitz.csRGB, pix)

                img_name = f"page{page_idx+1}_img{img_idx+1}.png"
                pix.save(os.path.join(output_dir, img_name))
                count += 1
                pix = None

            except Exception as e:
                print(f"    ⚠️ Warning: Image error on page {page_idx+1}: {e}")

    doc.close()
    return count

# TEXT & TABLE parser
def parse_chapter_content(pdf_path):
    sections = []
    current_section = {"section_title": "Introduction", "content_blocks": []}

    with pdfplumber.open(pdf_path) as pdf:
        all_chars = []
        for p in pdf.pages: all_chars.extend(p.chars)

        base_size = 10
        if all_chars:
            sizes = [c['size'] for c in all_chars]
            base_size = Counter(sizes).most_common(1)[0][0]

        for page in pdf.pages:
            # Extracting Tables
            tables = page.extract_tables()
            for table in tables:
                current_section["content_blocks"].append({
                    "type": "table",
                    "content": table
                })
           # Extract Text with Hierarchy
            text_lines = page.extract_text_lines()
            for line in text_lines:
                text = line['text'].strip()
                if not text: continue

                chars = line['chars']
                if chars:
                    avg_size = sum([c['size'] for c in chars]) / len(chars)
                    if avg_size > base_size * 1.1:
                        if current_section["content_blocks"]:
                            sections.append(current_section)
                        current_section = {
                            "section_title": text,
                            "content_blocks": []
                        }
                    else:
                        current_section["content_blocks"].append({
                            "type": "paragraph_text",
                            "content": text
                        })

    sections.append(current_section)
    return sections

# Performing/Executing Loop
print("Starting the Robust Extraction-")

for subject, input_dir in SUBJECTS.items():
    print(f"\n Subject: {subject}")
    output_dir = os.path.join(OUTPUT_BASE_DIR, subject)
    os.makedirs(output_dir, exist_ok=True)

    if not os.path.exists(input_dir):
        print(f"⚠️ Input folder missing: {input_dir}")
        continue

    files = sorted([f for f in os.listdir(input_dir) if f.endswith('.pdf')])

    for file in files:
        cid = file.replace(".pdf", "")
        print(f" 📄 Processing {cid}...", end=" ")

        pdf_path = os.path.join(input_dir, file)
        img_dir = os.path.join(output_dir, f"{cid}_images")

        try:
            img_count = extract_images_robust(pdf_path, img_dir)
            sections = parse_chapter_content(pdf_path)

            data = {
                "book": subject,
                "chapter": cid,
                "sections": sections
            }
            with open(os.path.join(output_dir, f"{cid}.json"), "w") as f:
                json.dump(data, f, indent=4)

            print(f"✅ Success ({img_count} Images)")

        except Exception as e:
            print(f"❌ Error: {e}")

print("\nAll Done!!!.")

🚀 Starting Final Robust Extraction...

📚 Subject: Mathematics
  📄 Processing answers&solutions... ✅ Success (26 Images)
  📄 Processing ch1...     ⚠️ Warning: Image error on page 1: code=4: pixmap must be grayscale or rgb to write as png
✅ Success (19 Images)
  📄 Processing ch10... ✅ Success (21 Images)
  📄 Processing ch11... ✅ Success (17 Images)
  📄 Processing ch12... ✅ Success (48 Images)
  📄 Processing ch13... 

Recognizing Text:  99%|█████████▊| 144/146 [13:03<00:10,  5.44s/it]


✅ Success (63 Images)
  📄 Processing ch14... ✅ Success (36 Images)
  📄 Processing ch2... ✅ Success (29 Images)
  📄 Processing ch3... ✅ Success (30 Images)
  📄 Processing ch4... ✅ Success (515 Images)
  📄 Processing ch5... ✅ Success (66 Images)
  📄 Processing ch6...     ⚠️ Warning: Image error on page 3: code=4: pixmap must be grayscale or rgb to write as png
    ⚠️ Warning: Image error on page 3: code=4: pixmap must be grayscale or rgb to write as png
    ⚠️ Warning: Image error on page 3: code=4: pixmap must be grayscale or rgb to write as png
✅ Success (58 Images)
  📄 Processing ch7...     ⚠️ Warning: Image error on page 1: code=4: pixmap must be grayscale or rgb to write as png
✅ Success (520 Images)
  📄 Processing ch8... ✅ Success (217 Images)
  📄 Processing ch9...     ⚠️ Warning: Image error on page 3: code=4: pixmap must be grayscale or rgb to write as png
✅ Success (507 Images)

📚 Subject: Science
  📄 Processing answers&solutions... ✅ Success (6 Images)
  📄 Processing ch1... 

✅ Success (64 Images)
  📄 Processing ch10... ✅ Success (25 Images)
  📄 Processing ch11... ✅ Success (49 Images)
  📄 Processing ch12... 

✅ Success (31 Images)
  📄 Processing ch13... ✅ Success (22 Images)
  📄 Processing ch2... 

✅ Success (51 Images)
  📄 Processing ch3... ✅ Success (52 Images)
  📄 Processing ch4... ✅ Success (54 Images)
  📄 Processing ch5... 

✅ Success (212 Images)
  📄 Processing ch6... ✅ Success (33 Images)
  📄 Processing ch7... 

✅ Success (127 Images)
  📄 Processing ch8... ✅ Success (93 Images)
  📄 Processing ch9... ✅ Success (55 Images)

🏁 DONE. Ready for Submission.


In [10]:
import json
import os

# Check a random chapter (e.g., Science Chapter 1)
file_path = "/content/drive/MyDrive/NCERT_Assignment/Output_Data/Science/ch1.json"

if os.path.exists(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)

    print(f" Book: {data['book']}")
    print(f" Chapter: {data['chapter']}")
    print(f" Total Sections: {len(data['sections'])}")

    # Print the first 3 content blocks of the first section to check quality
    first_section = data['sections'][0]
    print(f"\n--- Sample Content ({first_section['section_title']}) ---")
    for block in first_section['content_blocks'][:3]:
        print(f"[{block['type']}]: {block['content'][:100]}...")
else:
    print(" File not found. Check paths.")

 Book: Science
 Chapter: ch1
 Total Sections: 27

--- Sample Content (Introduction) ---
[paragraph_text]: “Facts are not science — as the dictionary is not literature.”...
[paragraph_text]: Martin H. Fischer...
